In [ ]:
import pyscf
from pyscf import scf, tools
import numpy
from pyscf_util.File.file_cmoao import ReadIn_Cmoao, Dump_Cmoao
from pyscf_util.misc.orb_comp_analysis import Analysis_MO_Component, print_dict_as_table
from pyscf_util.misc.mole import get_orbsym

GEOMETRY = """
Fe       0.000000        0.000000        0.000000
N1        3.734855       -0.000000        0.000000
N2       -0.000000        0.000000       -3.757531
N3       -3.734855        0.000000        0.000000
N4        0.000000        0.000000        3.757531
C21        5.325626       -0.000000        2.060368
C22        5.325626       -0.000000       -2.060368
C3        2.063203       -0.000000       -5.338287
C4       -2.063203        0.000000       -5.338287
C5       -5.325626        0.000000       -2.060368
C6       -5.325626        0.000000        2.060368
C7       -2.063203        0.000000        5.338287
C8        2.063203       -0.000000        5.338287
C9        7.929480       -0.000000        1.279912
C10        7.929480       -0.000000       -1.279912
C11        1.289738       -0.000000       -7.920220
C12      -1.289738        0.000000       -7.920220
C13       -7.929480        0.000000       -1.279912
C14       -7.929480        0.000000        1.279912
C15       -1.289738        0.000000        7.920220
C16        1.289738       -0.000000        7.920220
H21        9.531968       -0.000000        2.558311
H22        9.531968       -0.000000       -2.558311
H3        2.562091       -0.000000       -9.527243
H4       -2.562091        0.000000       -9.527243
H5       -9.531968        0.000000       -2.558311
H6       -9.531968        0.000000        2.558311
H7       -2.562091        0.000000        9.527243
H8        2.562091       -0.000000        9.527243
C17       4.563689       -0.000000       -4.551027
C18      -4.563689        0.000000       -4.551027
C19       -4.563689        0.000000        4.551027
C20        4.563689       -0.000000        4.551027
H9        6.019723       -0.000000       -6.000258
H10       -6.019723        0.000000       -6.000258
H11       -6.019723        0.000000        6.000258
H12        6.019723       -0.000000        6.000258
""" ## ？ 找不到 d2h ? 

GEOMETRY = """
Fe       0.000000        0.000000        0.000000
N        3.734855       -0.000000        0.000000
N       -0.000000        0.000000       -3.757531
N       -3.734855        0.000000        0.000000
N        0.000000        0.000000        3.757531
C        5.325626       -0.000000        2.060368
C        5.325626       -0.000000       -2.060368
C        2.063203       -0.000000       -5.338287
C       -2.063203        0.000000       -5.338287
C       -5.325626        0.000000       -2.060368
C       -5.325626        0.000000        2.060368
C       -2.063203        0.000000        5.338287
C        2.063203       -0.000000        5.338287
C        7.929480       -0.000000        1.279912
C        7.929480       -0.000000       -1.279912
C        1.289738       -0.000000       -7.920220
C       -1.289738        0.000000       -7.920220
C       -7.929480        0.000000       -1.279912
C       -7.929480        0.000000        1.279912
C       -1.289738        0.000000        7.920220
C        1.289738       -0.000000        7.920220
H        9.531968       -0.000000        2.558311
H        9.531968       -0.000000       -2.558311
H        2.562091       -0.000000       -9.527243
H       -2.562091        0.000000       -9.527243
H       -9.531968        0.000000       -2.558311
H       -9.531968        0.000000        2.558311
H       -2.562091        0.000000        9.527243
H        2.562091       -0.000000        9.527243
C        4.563689       -0.000000       -4.551027
C       -4.563689        0.000000       -4.551027
C       -4.563689        0.000000        4.551027
C        4.563689       -0.000000        4.551027
H        6.019723       -0.000000       -6.000258
H       -6.019723        0.000000       -6.000258
H       -6.019723        0.000000        6.000258
H        6.019723       -0.000000        6.000258
"""

In [ ]:
# ROHF #

Mol = pyscf.gto.Mole()
Mol.atom = GEOMETRY
Mol.basis = "cc-pvdz"
Mol.symmetry = "d2h"
Mol.spin = 4
Mol.charge = 0
Mol.verbose = 4
Mol.unit = "bohr"
Mol.build()

fep_rohf = ReadIn_Cmoao("FeP_ROHF", Mol.nao, Mol.nao)

# first (186 - 4) // 2 + 4 = 95 orbitals

# basis = {
#     "Fe":"cc-pvdz",
#     "C":"cc-pvdz",
#     "N":"cc-pvdz",
#     "H":"cc-pvdz",
# }

In [ ]:
# do SVD # 

from pyscf_util._atmMinCASOrb._orb_loader import (
    LoadAtmHFOrbAllInfo,
    LoadAtmHFOrbGivenType,
)

orb = LoadAtmHFOrbGivenType("Fe", 0, "cc-pvdz", with_sfx2c=False,orb_symbol="3d")

Fe3d_inFeP = numpy.zeros((Mol.nao, orb.shape[1]))
Fe3d_inFeP[:orb.shape[0], :] = orb

orbsym,_ = get_orbsym(Mol, Fe3d_inFeP)
# print(orbsym)
ovlp = Mol.intor("int1e_ovlp")
occ_mo = fep_rohf[:, :95] 
orbsym_occ,_ = get_orbsym(Mol, occ_mo)

print(orbsym)
print(orbsym_occ)

In [ ]:
# 找出 occ_mo 和 Fe3d_inFeP 中对称性标记为给定值的轨道
def filter_orbitals_by_symmetry(orbitals, orbsym, target_symmetry):
    indices = [i for i, sym in enumerate(orbsym) if sym == target_symmetry]
    return orbitals[:, indices], indices

occ_mo_new = numpy.zeros(occ_mo.shape)
start_loc = 0
end_loc = occ_mo_new.shape[1] - 5

for target_symmetry in [0,1,2,3]:

    # 给定对称性标记
    # target_symmetry = 0

    # 筛选 occ_mo 和 Fe3d_inFeP 中对称性为 target_symmetry 的轨道
    filtered_occ_mo, indices = filter_orbitals_by_symmetry(occ_mo, orbsym_occ, target_symmetry)
    filtered_Fe3d_inFeP, _ = filter_orbitals_by_symmetry(Fe3d_inFeP, orbsym, target_symmetry)

    # 计算两组轨道的重叠矩阵
    filtered_ovlp = filtered_occ_mo.T @ ovlp @ filtered_Fe3d_inFeP

    print(filtered_occ_mo.shape)

    # 对重叠矩阵进行 SVD 分解
    U, S, Vt = numpy.linalg.svd(filtered_ovlp, full_matrices=True, )
    print(S)
    # 将 occ_mo 中 SVD 较大的移动到最右端
    sorted_indices = numpy.argsort(-S)  # 按 SVD 值降序排列
    # print(sorted_indices)
    # print(U)
    # sorted_occ_mo = numpy.hstack([filtered_occ_mo[:, sorted_indices], occ_mo[:, filtered_occ_mo.shape[1]:]])
    updated_occ_mo = filtered_occ_mo @ U
    # 将 updated_occ_mo 的前 len(S) 列挪动到最后
    num_columns_to_move = len(S)
    # updated_occ_mo = numpy.hstack([updated_occ_mo[:, num_columns_to_move:], updated_occ_mo[:, :num_columns_to_move]])
    # occ_mo[:, indices] = updated_occ_mo

    occ_mo_new[:, start_loc:start_loc+updated_occ_mo.shape[1]-num_columns_to_move] = updated_occ_mo[:, num_columns_to_move:]
    occ_mo_new[:, end_loc:end_loc+num_columns_to_move] = updated_occ_mo[:, :num_columns_to_move]
    
    print(updated_occ_mo[:, :num_columns_to_move].T @ ovlp @ filtered_Fe3d_inFeP)
    print(start_loc, end_loc)
    
    assert(numpy.allclose(updated_occ_mo.T @ ovlp @ updated_occ_mo, numpy.identity((updated_occ_mo.shape[1]))))
    
    start_loc += updated_occ_mo.shape[1]-num_columns_to_move
    end_loc += num_columns_to_move

    # 更新 occ_mo
    # occ_mo = sorted_occ_mo

for target_symmetry in [4,5,6,7]:
    filtered_occ_mo, indices = filter_orbitals_by_symmetry(occ_mo, orbsym_occ, target_symmetry)
    occ_mo_new[:, start_loc:start_loc+filtered_occ_mo.shape[1]] = filtered_occ_mo
    # occ_mo_new[:, end_loc:end_loc+num_columns_to_move] = updated_occ_mo[:, :num_columns_to_move]
    start_loc += filtered_occ_mo.shape[1]

In [ ]:
fep_rohf[:, :95] = occ_mo_new

Dump_Cmoao("FeP_ROHF_InitGuess", fep_rohf)

In [ ]:
# # check #
# print(occ_mo_new.T @ ovlp @ occ_mo_new)
# assert(numpy.allclose(occ_mo_new.T @ ovlp @ occ_mo_new, numpy.identity((95))))
# print(occ_mo_new[:, -5:].T @ ovlp @ Fe3d_inFeP)